### Look at Portland 2024 CVRs for city council STV election and get them into a nicer format.
### This notebook: District 3

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

import numpy as np

import maup
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
maup.progress.enabled = True

from tqdm import tqdm

In [2]:
D3_df = pd.read_csv("./Data/Cast Vote Records/City_of_Portland__Councilor__District_3_2024_11_29_17_26_12.cvr.csv")

In [3]:
D3_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,"Choice_58_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_59_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Philippe Knab:NON","Choice_60_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Harrison Kass:NON","Choice_61_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Kezia Wanner:NON","Choice_62_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_63_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Melodie Beirwagen:NON","Choice_64_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland,

### Note that this is all the voters in the city, not just in D3.

In [4]:
list(D3_df.columns)

['RowNumber',
 'BoxID',
 'BoxPosition',
 'BallotID',
 'PrecinctID',
 'BallotStyleID',
 'PrecinctStyleName',
 'ScanComputerName',
 'Status',
 'Remade',
 'Choice_58_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Tiffany Koyama Lane:NON',
 'Choice_58_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Tiffany Koyama Lane:NON',
 'Choice_58_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Tiffany Koyama Lane:NON',
 'Choice_58_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Tiffany Koyama Lane:NON',
 'Choice_58_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Tiffany Koyama Lane:NON',
 'Choice_58_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Tiffany Koyama Lane:NON',
 'Choice_59_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Philippe Knab:NON',
 'Choice_59_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Philippe Knab:NON',
 'Choice_59_1:City of Portland, Councilor, Di

### Here are all the candidates that the CVR contains rankings for:

In [5]:
D3_candidates = [
    'Steve Novick',
    'Angelita Morillo',
    'Tiffany Koyama Lane',
    'Kezia Wanner',
    'Rex Burkholder',
    'Jesse Cornett',
    'Harrison Kass',
    'Daniel DeMelo',
    'Philippe Knab',
    'Sandeep Bali',
    'Cristal Azul Otero',
    'Jonathan (Jon) Walker',
    'Chris Flanary',
    'Melodie Beirwagen',
    'Matthew (Matt) Anderson',
    'Ahlam K Osman',
    'Luke Zak',
    'Heart Free Pham',
    'Brian Conley',
    'Terry Parker',
    'Dan Gilk',
    'Christopher Brummer',
    'Uncertified Write In',
    'John Sweeney',
    'Kelly Janes (KJ)',
    'Theo Hathaway Saner',
    'Patrick Hilton',
    'Jaclyn Smith-Moore',
    "David O'Connor",
    'Kenneth (Kent) R Landgraver III',
    'Clifford Higgins',
    'Write-in-126',
    'Write-in-127',
    'Write-in-128'
]

### Collect columns where 1st place, etc. votes are tallied so we can look for overvotes/undervotes while converting to a better format.

In [6]:
rank1_cols = [col for col in D3_df.columns if '1:Number' in col]
rank2_cols = [col for col in D3_df.columns if '2:Number' in col]
rank3_cols = [col for col in D3_df.columns if '3:Number' in col]
rank4_cols = [col for col in D3_df.columns if '4:Number' in col]
rank5_cols = [col for col in D3_df.columns if '5:Number' in col]
rank6_cols = [col for col in D3_df.columns if '6:Number' in col]

all_rank_cols = rank1_cols + rank2_cols + rank3_cols + rank4_cols + rank5_cols + rank6_cols

### This dataset has rows for ALL Portland voters; let's trim it down to voters who cast at least one vote.

In [7]:
D3_voters_df = D3_df[D3_df[all_rank_cols].sum(axis=1) > 0].reset_index(drop=True)

In [8]:
D3_voters_df 

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,"Choice_58_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_59_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Philippe Knab:NON","Choice_60_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Harrison Kass:NON","Choice_61_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Kezia Wanner:NON","Choice_62_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_63_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Melodie Beirwagen:NON","Choice_64_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland,

In [9]:
sorted(list(set(D3_voters_df['PrecinctStyleName'])))

['4101-2',
 '4104-1',
 '4105-2',
 '4201-1',
 '4202-1',
 '4203-1',
 '4204-1',
 '4205-1',
 '4206-1',
 '4207-1',
 '4208-1',
 '4501-2',
 '4502-1',
 '4503-1',
 '4505-1',
 '4506-1',
 '4507-1',
 '4508-2',
 '4601-1',
 '4602-1',
 '4603-1',
 '4604-1',
 '4605-1',
 '4606-1',
 'CLACK-114-2']

### So this file DOES include voters in Clackamas County.  (Washington County will only be relevant in D4.)

### Add a column for each raw ranking:

In [10]:
D3_voters_df.insert(10, "Rank1 raw", None)
D3_voters_df.insert(11, "Rank2 raw", None)
D3_voters_df.insert(12, "Rank3 raw", None)
D3_voters_df.insert(13, "Rank4 raw", None)
D3_voters_df.insert(14, "Rank5 raw", None)
D3_voters_df.insert(15, "Rank6 raw", None)


### For each voter and each rank, record whether the voter overvoted or undervoted in that rank, or if they voted for exactly one candidate in that rank, record that candidate.

In [11]:
for i in tqdm(D3_voters_df.index):
    
    rank1_votes = D3_voters_df.loc[i, rank1_cols].sum()
    if rank1_votes == 0:
        D3_voters_df.loc[i, 'Rank1 raw'] = "skipped"
    elif rank1_votes > 1:
        D3_voters_df.loc[i, 'Rank1 raw'] = "overvote"
    elif rank1_votes == 1:
        this_vote_col = [col for col in rank1_cols if D3_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D3_candidates if name in this_vote_col][0]
        D3_voters_df.loc[i, 'Rank1 raw'] = this_vote

    rank2_votes = D3_voters_df.loc[i, rank2_cols].sum()
    if rank2_votes == 0:
        D3_voters_df.loc[i, 'Rank2 raw'] = "skipped"
    elif rank2_votes > 1:
        D3_voters_df.loc[i, 'Rank2 raw'] = "overvote"
    elif rank2_votes == 1:
        this_vote_col = [col for col in rank2_cols if D3_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D3_candidates if name in this_vote_col][0]
        D3_voters_df.loc[i, 'Rank2 raw'] = this_vote

    rank3_votes = D3_voters_df.loc[i, rank3_cols].sum()
    if rank3_votes == 0:
        D3_voters_df.loc[i, 'Rank3 raw'] = "skipped"
    elif rank3_votes > 1:
        D3_voters_df.loc[i, 'Rank3 raw'] = "overvote"
    elif rank3_votes == 1:
        this_vote_col = [col for col in rank3_cols if D3_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D3_candidates if name in this_vote_col][0]
        D3_voters_df.loc[i, 'Rank3 raw'] = this_vote

    rank4_votes = D3_voters_df.loc[i, rank4_cols].sum()
    if rank4_votes == 0:
        D3_voters_df.loc[i, 'Rank4 raw'] = "skipped"
    elif rank4_votes > 1:
        D3_voters_df.loc[i, 'Rank4 raw'] = "overvote"
    elif rank4_votes == 1:
        this_vote_col = [col for col in rank4_cols if D3_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D3_candidates if name in this_vote_col][0]
        D3_voters_df.loc[i, 'Rank4 raw'] = this_vote

    rank5_votes = D3_voters_df.loc[i, rank5_cols].sum()
    if rank5_votes == 0:
        D3_voters_df.loc[i, 'Rank5 raw'] = "skipped"
    elif rank5_votes > 1:
        D3_voters_df.loc[i, 'Rank5 raw'] = "overvote"
    elif rank5_votes == 1:
        this_vote_col = [col for col in rank5_cols if D3_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D3_candidates if name in this_vote_col][0]
        D3_voters_df.loc[i, 'Rank5 raw'] = this_vote

    rank6_votes = D3_voters_df.loc[i, rank6_cols].sum()
    if rank6_votes == 0:
        D3_voters_df.loc[i, 'Rank6 raw'] = "skipped"
    elif rank6_votes > 1:
        D3_voters_df.loc[i, 'Rank6 raw'] = "overvote"
    elif rank6_votes == 1:
        this_vote_col = [col for col in rank6_cols if D3_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D3_candidates if name in this_vote_col][0]
        D3_voters_df.loc[i, 'Rank6 raw'] = this_vote


100%|████████████████████████████████████| 85115/85115 [03:21<00:00, 423.33it/s]


In [12]:
D3_voters_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,Rank1 raw,Rank2 raw,Rank3 raw,Rank4 raw,Rank5 raw,Rank6 raw,"Choice_58_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_59_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Philippe Knab:NON","Choice_60_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Harrison Kass:NON","Choice_61_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Kezia Wanner:NON","Choice_62_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_63_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Melodie Beirwagen:NON","Choice_64_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:4:Number o

### Next, clean the votes for VoteKit according to Portland's rules: overvotes and undervotes are both skipped and ballots are transferred to the next ranking.
### It appears that the categories 'Write-in-126', 'Write-in-127', and 'Write-in-128' are ALSO skipped and ballots are transferred to the next ranking.  (Equivalently, these "candidates" were eliminated first in a "Round 0" that occurred before the published tabulation rounds.)

In [13]:
D3_voters_df.insert(16, "Rank1 cleaned", None)
D3_voters_df.insert(17, "Rank2 cleaned", None)
D3_voters_df.insert(18, "Rank3 cleaned", None)
D3_voters_df.insert(19, "Rank4 cleaned", None)
D3_voters_df.insert(20, "Rank5 cleaned", None)
D3_voters_df.insert(21, "Rank6 cleaned", None)


### While we do this, save the cleaned ballots to a list to run the election with.

In [14]:
D3_candidates_R1 = D3_candidates[:-3]
D3_candidates_R1

['Steve Novick',
 'Angelita Morillo',
 'Tiffany Koyama Lane',
 'Kezia Wanner',
 'Rex Burkholder',
 'Jesse Cornett',
 'Harrison Kass',
 'Daniel DeMelo',
 'Philippe Knab',
 'Sandeep Bali',
 'Cristal Azul Otero',
 'Jonathan (Jon) Walker',
 'Chris Flanary',
 'Melodie Beirwagen',
 'Matthew (Matt) Anderson',
 'Ahlam K Osman',
 'Luke Zak',
 'Heart Free Pham',
 'Brian Conley',
 'Terry Parker',
 'Dan Gilk',
 'Christopher Brummer',
 'Uncertified Write In',
 'John Sweeney',
 'Kelly Janes (KJ)',
 'Theo Hathaway Saner',
 'Patrick Hilton',
 'Jaclyn Smith-Moore',
 "David O'Connor",
 'Kenneth (Kent) R Landgraver III',
 'Clifford Higgins']

In [15]:
cleaned_ballots_R1 = []

for i in tqdm(D3_voters_df.index):
    this_ballot_raw = D3_voters_df.loc[i, ['Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw', 'Rank5 raw', 'Rank6 raw']]
    
    this_ballot_cleaned = [vote for vote in this_ballot_raw if vote in D3_candidates_R1]

    if len(this_ballot_cleaned) > 0:
        cleaned_ballots_R1.append(this_ballot_cleaned)
    
    if len(this_ballot_cleaned) < 6:
        fill_len = 6 - len(this_ballot_cleaned)
        this_ballot_cleaned = this_ballot_cleaned + ["skipped"]*fill_len

    D3_voters_df.loc[i, "Rank1 cleaned"] = this_ballot_cleaned[0]
    D3_voters_df.loc[i, "Rank2 cleaned"] = this_ballot_cleaned[1]
    D3_voters_df.loc[i, "Rank3 cleaned"] = this_ballot_cleaned[2]
    D3_voters_df.loc[i, "Rank4 cleaned"] = this_ballot_cleaned[3]
    D3_voters_df.loc[i, "Rank5 cleaned"] = this_ballot_cleaned[4]
    D3_voters_df.loc[i, "Rank6 cleaned"] = this_ballot_cleaned[5]
    

100%|███████████████████████████████████| 85115/85115 [00:37<00:00, 2279.31it/s]


### How many first-round votes did each candidate get?

In [16]:
round_1_votes = {}
for name in D3_candidates_R1:
    round_1_votes[name] = 0

for ballot in cleaned_ballots_R1:
    round_1_votes[ballot[0]] += 1

In [17]:
for name in D3_candidates_R1:
    print(name, round_1_votes[name])


Steve Novick 20457
Angelita Morillo 16399
Tiffany Koyama Lane 16320
Kezia Wanner 5313
Rex Burkholder 3951
Jesse Cornett 3860
Harrison Kass 2786
Daniel DeMelo 1571
Philippe Knab 1551
Sandeep Bali 1408
Cristal Azul Otero 1405
Jonathan (Jon) Walker 1322
Chris Flanary 1242
Melodie Beirwagen 1132
Matthew (Matt) Anderson 755
Ahlam K Osman 709
Luke Zak 548
Heart Free Pham 546
Brian Conley 506
Terry Parker 375
Dan Gilk 330
Christopher Brummer 263
Uncertified Write In 248
John Sweeney 236
Kelly Janes (KJ) 225
Theo Hathaway Saner 219
Patrick Hilton 194
Jaclyn Smith-Moore 194
David O'Connor 174
Kenneth (Kent) R Landgraver III 172
Clifford Higgins 104


### Excellent - we got the algorithm right!

### How many voters voted in the first round?

In [18]:
len(cleaned_ballots_R1)

84515

### Just for kicks, check the round 2 tallies too.

In [19]:
D3_candidates_R2 = D3_candidates_R1[:-1]

In [20]:
D3_candidates_R2

['Steve Novick',
 'Angelita Morillo',
 'Tiffany Koyama Lane',
 'Kezia Wanner',
 'Rex Burkholder',
 'Jesse Cornett',
 'Harrison Kass',
 'Daniel DeMelo',
 'Philippe Knab',
 'Sandeep Bali',
 'Cristal Azul Otero',
 'Jonathan (Jon) Walker',
 'Chris Flanary',
 'Melodie Beirwagen',
 'Matthew (Matt) Anderson',
 'Ahlam K Osman',
 'Luke Zak',
 'Heart Free Pham',
 'Brian Conley',
 'Terry Parker',
 'Dan Gilk',
 'Christopher Brummer',
 'Uncertified Write In',
 'John Sweeney',
 'Kelly Janes (KJ)',
 'Theo Hathaway Saner',
 'Patrick Hilton',
 'Jaclyn Smith-Moore',
 "David O'Connor",
 'Kenneth (Kent) R Landgraver III']

In [21]:
cleaned_ballots_R2 = []

for ballot in tqdm(cleaned_ballots_R1):
    new_ballot = [name for name in ballot if name in D3_candidates_R2]
    if len(new_ballot) > 0:
        cleaned_ballots_R2.append(new_ballot)

100%|█████████████████████████████████| 84515/84515 [00:00<00:00, 910446.93it/s]


### Round 2 votes:

In [22]:
round_2_votes = {}
for name in D3_candidates_R2:
    round_2_votes[name] = 0

for ballot in cleaned_ballots_R2:
    round_2_votes[ballot[0]] += 1

In [23]:
for name in D3_candidates_R2:
    print(name, round_2_votes[name])


Steve Novick 20468
Angelita Morillo 16411
Tiffany Koyama Lane 16326
Kezia Wanner 5314
Rex Burkholder 3958
Jesse Cornett 3863
Harrison Kass 2791
Daniel DeMelo 1573
Philippe Knab 1553
Sandeep Bali 1409
Cristal Azul Otero 1405
Jonathan (Jon) Walker 1324
Chris Flanary 1248
Melodie Beirwagen 1133
Matthew (Matt) Anderson 756
Ahlam K Osman 716
Luke Zak 551
Heart Free Pham 548
Brian Conley 507
Terry Parker 377
Dan Gilk 331
Christopher Brummer 264
Uncertified Write In 249
John Sweeney 239
Kelly Janes (KJ) 226
Theo Hathaway Saner 220
Patrick Hilton 198
Jaclyn Smith-Moore 198
David O'Connor 176
Kenneth (Kent) R Landgraver III 174


### These are correct too!

### So, let's create two datasets to save: one with the raw data and one with the cleaned data.  Keep all the intro columns for the raw data, but not for the cleaned data.

In [24]:
D3_voters_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,Rank1 raw,Rank2 raw,Rank3 raw,Rank4 raw,Rank5 raw,Rank6 raw,Rank1 cleaned,Rank2 cleaned,Rank3 cleaned,Rank4 cleaned,Rank5 cleaned,Rank6 cleaned,"Choice_58_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_58_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Tiffany Koyama Lane:NON","Choice_59_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Philippe Knab:NON","Choice_59_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Philippe Knab:NON","Choice_60_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Harrison Kass:NON","Choice_60_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Harrison Kass:NON","Choice_61_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Kezia Wanner:NON","Choice_61_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Kezia Wanner:NON","Choice_62_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_62_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Jonathan (Jon) Walker:NON","Choice_63_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:3:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:4:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:5:Number of Winners 3:Melodie Beirwagen:NON","Choice_63_1:City of Portland, Councilor, District 3:6:Number of Winners 3:Melodie Beirwagen:NON","Choice_64_1:City of Portland, Councilor, District 3:1:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:2:Number of Winners 3:Sandeep Bali:NON","Choice_64_1:City of Portland, Councilor, District 3:3:Number of Winners

In [25]:
D3_voters_df.columns[0:16]

Index(['RowNumber', 'BoxID', 'BoxPosition', 'BallotID', 'PrecinctID',
       'BallotStyleID', 'PrecinctStyleName', 'ScanComputerName', 'Status',
       'Remade', 'Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw',
       'Rank5 raw', 'Rank6 raw'],
      dtype='object')

In [26]:
D3_raw_df = D3_voters_df[D3_voters_df.columns[0:16]]

In [27]:
D3_voters_df.columns[16:22]

Index(['Rank1 cleaned', 'Rank2 cleaned', 'Rank3 cleaned', 'Rank4 cleaned',
       'Rank5 cleaned', 'Rank6 cleaned'],
      dtype='object')

In [28]:
D3_cleaned_df = D3_voters_df[D3_voters_df.columns[16:22]]

### The raw file is good to go; I just want to rename the ranking columns.

In [29]:
D3_raw_df = D3_raw_df.rename(columns = {
    'Rank1 raw' : 'rank1',
    'Rank2 raw' : 'rank2',
    'Rank3 raw' : 'rank3',
    'Rank4 raw' : 'rank4',
    'Rank5 raw' : 'rank5',
    'Rank6 raw' : 'rank6'
})

In [30]:
D3_raw_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,rank1,rank2,rank3,rank4,rank5,rank6
0,1,RCV-0001,1,RCV-0001+10003,26,3,4506-1,ScanStation6,0,0,Rex Burkholder,Philippe Knab,skipped,skipped,skipped,skipped
1,7,RCV-0001,7,RCV-0001+10015,71,3,4204-1,ScanStation6,0,0,Tiffany Koyama Lane,Rex Burkholder,Angelita Morillo,Jesse Cornett,Theo Hathaway Saner,Cristal Azul Otero
2,26,RCV-0001,26,RCV-0001+10053,21,3,4202-1,ScanStation6,0,0,Angelita Morillo,Tiffany Koyama Lane,Steve Novick,Rex Burkholder,skipped,skipped
3,28,RCV-0001,28,RCV-0001+10057,68,3,4601-1,ScanStation6,0,0,David O'Connor,Terry Parker,Harrison Kass,John Sweeney,skipped,skipped
4,29,RCV-0001,29,RCV-0001+10059,26,3,4506-1,ScanStation6,0,0,Angelita Morillo,Tiffany Koyama Lane,Ahlam K Osman,skipped,skipped,skipped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85110,332951,RCV-1944,12,RCV-1944+10025,71,3,4204-1,ScanStation6,0,0,Luke Zak,Chris Flanary,Heart Free Pham,Cristal Azul Otero,Brian Conley,Jonathan (Jon) Walker
85111,332953,RCV-1944,14,RCV-1944+10029,3,3,4207-1,ScanStation6,0,0,Tiffany Koyama Lane,Jesse Cornett,Cristal Azul Otero,Heart Free Pham,Philippe Knab,Chris Flanary
85112,332961,RCV-1944,22,RCV-1944+10045,71,3,4204-1,ScanStation6,0,0,Steve Novick,Angelita Morillo,Cristal Azul Otero,Philippe Knab,Tiffany Koyama Lane,Rex Burkholder
85113,332962,RCV-1944,23,RCV-1944+10047,22,3,4105-2,ScanStation6,0,0,Tiffany Koyama Lane,Heart Free Pham,Jaclyn Smith-Moore,skipped,skipped,skipped


In [31]:
D3_raw_df.to_csv("./Data/Cast Vote Records/Cleaned CVRs/Portland_D3_Raw.csv", index = False)

### For the cleaned file, I'd like to remove voters who no longer have any votes. (These are the ones who only voted for 'Write-in-126', 'Write-in-127', and/or 'Write-in-128'.)  
### Check that this produces the correct number of first-round votes:

In [32]:
len(D3_cleaned_df[D3_cleaned_df['Rank1 cleaned'] != 'skipped'])

84515

In [33]:
D3_cleaned_df = D3_cleaned_df[D3_cleaned_df['Rank1 cleaned'] != 'skipped'].reset_index(drop="True")

### Rename the columns:

In [34]:
D3_cleaned_df = D3_cleaned_df.rename(columns = {
    'Rank1 cleaned' : 'rank1',
    'Rank2 cleaned' : 'rank2',
    'Rank3 cleaned' : 'rank3',
    'Rank4 cleaned' : 'rank4',
    'Rank5 cleaned' : 'rank5',
    'Rank6 cleaned' : 'rank6'
})

In [35]:
D3_cleaned_df

,rank1,rank2,rank3,rank4,rank5,rank6
0,Rex Burkholder,Philippe Knab,skipped,skipped,skipped,skipped
1,Tiffany Koyama Lane,Rex Burkholder,Angelita Morillo,Jesse Cornett,Theo Hathaway Saner,Cristal Azul Otero
2,Angelita Morillo,Tiffany Koyama Lane,Steve Novick,Rex Burkholder,skipped,skipped
3,David O'Connor,Terry Parker,Harrison Kass,John Sweeney,skipped,skipped
4,Angelita Morillo,Tiffany Koyama Lane,Ahlam K Osman,skipped,skipped,skipped
...,...,...,...,...,...,...
84510,Luke Zak,Chris Flanary,Heart Free Pham,Cristal Azul Otero,Brian Conley,Jonathan (Jon) Walker
84511,Tiffany Koyama Lane,Jesse Cornett,Cristal Azul Otero,Heart Free Pham,Philippe Knab,Chris Flanary
84512,Steve Novick,Angelita Morillo,Cristal Azul Otero,Philippe Knab,Tiffany Koyama Lane,Rex Burkholder
84513,Tiffany Koyama Lane,Heart Free Pham,Jaclyn Smith-Moore,skipped,skipped,skipped


In [36]:
D3_cleaned_df.to_csv("./Data/Cast Vote Records/Cleaned CVRs/Portland_D3_Cleaned.csv", index = False)